In [3]:
# %load preprocess_resume.py
#!/usr/bin/env python
from tika import parser
import os
import sys
import numpy as np
import re
import pandas as pd
# import magic

class preprocess_resume(object):
    def __init__(self, document_path, tika_server = "http://localhost:9998/"):
        self.document_path = document_path
        assert os.path.isfile(self.document_path)
        self.tika_server = tika_server

    def parse_pdf_to_txt(self):
        os.system("pdftotext " + self.document_path+ " " + ((self.output_path+'/%s_temp.txt') % os.path.splitext(document)[0]))

    def parse_tika(self):
        parsed = parser.from_file(self.document_path, self.tika_server)['content']
        parsed = [x.strip() for x in parsed.split("\n") if x.strip() != u""]
        return parsed
        
    def check_below(self, n_row, dta, i):
        result = str()
        for row in range(n_row):
            try:
                detect = re.findall(ur'[\u4e00-\u9fff]+', dta[i+row])
                if detect != []:
                    result = dta[i]
            except: 
                pass
        if len(result) == 0:
            return('')
        else: 
            return(result)
    
    def del_en(self, rows):
        output_list = []
        for i in range(len(rows)):
            if re.findall(ur'[\u4e00-\u9fff]+', rows[i]):
                output_list.append(rows[i])
            elif re.findall(ur'@', rows[i]):
                output_list.append(rows[i])
            elif re.findall(ur'[0-9]+', rows[i]):
                output_list.append(self.check_below(3,rows,i)) #? wt
            else: 
                pass
        return output_list

    def reformat_doc(self, content_list):
        split_re = u"；|、|\||\/|\ufe31|\u3001|\uff1a|\uff0c|;|】|【|，|。|！|？|：|\\n|,|:|\\s+|\\t+|[　]+"
        content_split = re.split(split_re, "\n".join(content_list))
        new_content_split = []
        cache_list = []
        for item in content_split:
            if len(item.strip()) > 1:#? wt
                if len(cache_list) != 0:
                    new_content_split.append("".join(cache_list))
                    cache_list = []
                new_content_split.append(item)
            elif len(item.strip()) == 1:
                cache_list.append(item)
        if len(cache_list) != 0:
            new_content_split.append("".join(cache_list))
        return new_content_split

    def process(self):
        try:
            content_list = self.reformat_doc(self.del_en(self.parse_tika()))
            self.content_df = pd.DataFrame(content_list, columns = ["text"])
            file_name = os.path.basename(self.document_path)
            n_row = len(self.content_df)
            self.content_df["resume_id"] = [file_name for x in range(n_row)]
            self.content_df["row"] = range(1, n_row+1)
            return self.content_df
        except Exception as e:
            print e
            print self.document_path
            return None

    def write(self, output_path):
        output_data = self.content_df
        output_data["output_text"] = output_data["text"].map(lambda x: x.encode("UTF-8"))
        output_data[["output_text"]].to_csv(output_path, header = None, index = None, sep = "\t", mode = 'wb')
        


# if __name__ == "__main__":
#   temp = preprocess_resume('/Users/Kay/Desktop/testusjackal', '/Users/Kay/Desktop/test-txt', "bulk")
#   temp.process()

if __name__ == "__main__":#wt
    temp = preprocess_resume('C:/Users/wangt/Desktop/testpic.jpg')
    content_list = temp.process()
    
    #content_list.to_csv('names', encoding='utf-8')
    

'NoneType' object has no attribute 'split'
C:/Users/wangt/Desktop/testpic.jpg


In [2]:
content_list

,text,resume_id,row
0,李永伟,test.pdf,1
1,男（1987）,test.pdf,2
2,广东省,test.pdf,3
3,深圳市,test.pdf,4
4,南山区,test.pdf,5
5,+86,test.pdf,6
6,139-2466-9071,test.pdf,7
7,13924669071@139.com,test.pdf,8
8,求职意向,test.pdf,9
9,嵌入式软硬件方向,test.pdf,10


In [3]:
print("test")

test


In [3]:
# %load generate_feature.py
"""
Created on 2017-05-03
@author: Guang
Generate feature for data_df
"""

import pandas as pd
import jieba
import re
from regex.time_reg import *
import sys
#wt reload(sys)
#wt sys.setdefaultencoding( "utf-8" )
stdout = sys.stdout #wt the following 4
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout
import pynlpir

class generate_feature(object):
    def __init__(self, data_df, stopword_path = "data/stop_words.txt"):
        self.stoplist = set(open(stopword_path).read().decode("utf8","ignore").split("\n"))
        self.data_df = data_df

    def add_new_words(self):
        self.data_df['new_text'] = self.data_df["text"].map(lambda x: ("".join(re.findall(ur'[a-zA-Z\u4e00-\u9fff]+', x))).strip())

    def add_token_words(self):
        self.data_df['token_words'] = self.data_df['new_text'].map(lambda x: [y for y in jieba.lcut(x, cut_all=False)])

    def add_time(self):
        self.data_df['contain_date'] = self.data_df['text'].str.contains(u"((?:19|20)\d{2})([-./\u5e74]?)((?:[0-1]?|1)[0-9])?([-./\u6708]?)((?:[0-3]?|[1-3])?[0-9])?(?:$|\D)").map(lambda x: True if x else False)

    def add_row_length(self):
        def _cal_length(text):
            if len(text) <= 5:
                return "5_char_less"
            elif len(text) >=15:
                return "15_char_more"
            else:
                return "15_char_less"
        self.data_df['text_length'] = self.data_df['text'].map(lambda x: _cal_length(x))
        self.data_df['text_length_previous'] = self.data_df.groupby('resume_id')['text_length']. \
            shift(periods=1).fillna('')
        self.data_df['text_length_next'] = self.data_df.groupby('resume_id')['text_length'].\
            shift(periods=-1).fillna('')

    def add_row_position(self):
        def _cal_position(position):
            if position <= 0.25:
                return "position_1"
            elif position <= 0.5:
                return "position_2"
            elif position <= 0.75:
                return "position_3"
            else:
                return "position_4"
        self.data_df['row_position'] = self.data_df.groupby(['resume_id'])['row'].transform(lambda x: x / x.max())
        self.data_df['row_position'] = self.data_df['row_position'].map(lambda x: _cal_position(x))

    def add_pos(self):
        pynlpir.open() #tell NLPIR to open the data files and initialize the API
        def _cal_pos(row):
            try:
                segments = pynlpir.segment(row, pos_names='all')
                return segments
            except Exception as e:
                return [(row, "None")]
        # remove '\u3000' in uncut resume
        # self.data_df["pos_text"] = self.data_df["new_text"].map(lambda x: x.replace(u"\u3000", ""))
        # self.data_df['pos'] = self.data_df["pos_text"].map(lambda x: _cal_pos(x))
        self.data_df['pos'] = self.data_df["new_text"].map(lambda x: _cal_pos(x))
        self.data_df['token_words'] = self.data_df['pos'].map(lambda x: [y[0] for y in x])
        pynlpir.close()

    def add_person_name(self):
        def _cal_person_name(row):
            result = [x[0] for x in row["pos"] if x[1] in [u'noun:personal name']]
            return [result, float(len("".join(result)))/len(row["text"])]
        self.data_df["person_name"] = self.data_df.apply(lambda x: _cal_person_name(x), axis = 1)

    def add_location(self):
        def _cal_location(row):
            result = [x[0] for x in row["pos"] if x[1] in [u'noun:toponym', u'noun:toponym:transcribed toponym']]
            return [result, float(len("".join(result)))/len(row["text"])]
        self.data_df["location"] = self.data_df.apply(lambda x: _cal_location(x), axis = 1)

    def add_crf_feature(self):
        self.add_new_words()
        # self.add_token_words()
        self.add_time()
        self.add_row_length()
        self.add_row_position()
        # self.data_df['crf_token_word'] = self.data_df['token_words'].map(lambda x:
        #     [y for y in x if y not in self.stoplist])
        self.add_pos()
        self.data_df['crf_token_word'] = self.data_df['pos'].map(lambda x: [segment[0] + ' '\
            + str(segment[1]) for segment in x])
        self.data_df['crf_feature'] = self.data_df.apply(lambda x: x["crf_token_word"] + \
            ["current: " + x["text_length"], \
            "previous: " + x["text_length_previous"], \
            "next: " + x["text_length_next"],
            x["row_position"],
            ("contain_date" if x["contain_date"] else "not_contain_date")], axis=1)

In [46]:
print("test1")

In [4]:
gf = generate_feature(content_list)

In [5]:
gf.data_df

,text,resume_id,row
0,李永伟,test.pdf,1
1,男（1987）,test.pdf,2
2,广东省,test.pdf,3
3,深圳市,test.pdf,4
4,南山区,test.pdf,5
5,+86,test.pdf,6
6,139-2466-9071,test.pdf,7
7,13924669071@139.com,test.pdf,8
8,求职意向,test.pdf,9
9,嵌入式软硬件方向,test.pdf,10


In [36]:
gf.add_new_words()

In [37]:
gf.data_df

,text,resume_id,row,new_text
0,李永伟,test.pdf,1,李永伟
1,男（1987）,test.pdf,2,男
2,广东省,test.pdf,3,广东省
3,深圳市,test.pdf,4,深圳市
4,南山区,test.pdf,5,南山区
5,+86,test.pdf,6,
6,139-2466-9071,test.pdf,7,
7,13924669071@139.com,test.pdf,8,com
8,求职意向,test.pdf,9,求职意向
9,嵌入式软硬件方向,test.pdf,10,嵌入式软硬件方向


In [38]:
gf.add_time()

In [39]:
gf.data_df

,text,resume_id,row,new_text,contain_date
0,李永伟,test.pdf,1,李永伟,False
1,男（1987）,test.pdf,2,男,True
2,广东省,test.pdf,3,广东省,False
3,深圳市,test.pdf,4,深圳市,False
4,南山区,test.pdf,5,南山区,False
5,+86,test.pdf,6,,False
6,139-2466-9071,test.pdf,7,,False
7,13924669071@139.com,test.pdf,8,com,False
8,求职意向,test.pdf,9,求职意向,False
9,嵌入式软硬件方向,test.pdf,10,嵌入式软硬件方向,False


In [40]:
gf.add_row_length()

In [41]:
gf.data_df

,text,resume_id,row,new_text,contain_date,text_length,text_length_previous,text_length_next
0,李永伟,test.pdf,1,李永伟,False,5_char_less,,15_char_less
1,男（1987）,test.pdf,2,男,True,15_char_less,5_char_less,5_char_less
2,广东省,test.pdf,3,广东省,False,5_char_less,15_char_less,5_char_less
3,深圳市,test.pdf,4,深圳市,False,5_char_less,5_char_less,5_char_less
4,南山区,test.pdf,5,南山区,False,5_char_less,5_char_less,5_char_less
5,+86,test.pdf,6,,False,5_char_less,5_char_less,15_char_less
6,139-2466-9071,test.pdf,7,,False,15_char_less,5_char_less,15_char_more
7,13924669071@139.com,test.pdf,8,com,False,15_char_more,15_char_less,5_char_less
8,求职意向,test.pdf,9,求职意向,False,5_char_less,15_char_more,15_char_less
9,嵌入式软硬件方向,test.pdf,10,嵌入式软硬件方向,False,15_char_less,5_char_less,5_char_less


In [42]:
gf.add_row_position()

In [43]:
gf.data_df

,text,resume_id,row,new_text,contain_date,text_length,text_length_previous,text_length_next,row_position
0,李永伟,test.pdf,1,李永伟,False,5_char_less,,15_char_less,position_1
1,男（1987）,test.pdf,2,男,True,15_char_less,5_char_less,5_char_less,position_1
2,广东省,test.pdf,3,广东省,False,5_char_less,15_char_less,5_char_less,position_1
3,深圳市,test.pdf,4,深圳市,False,5_char_less,5_char_less,5_char_less,position_1
4,南山区,test.pdf,5,南山区,False,5_char_less,5_char_less,5_char_less,position_1
5,+86,test.pdf,6,,False,5_char_less,5_char_less,15_char_less,position_1
6,139-2466-9071,test.pdf,7,,False,15_char_less,5_char_less,15_char_more,position_1
7,13924669071@139.com,test.pdf,8,com,False,15_char_more,15_char_less,5_char_less,position_1
8,求职意向,test.pdf,9,求职意向,False,5_char_less,15_char_more,15_char_less,position_1
9,嵌入式软硬件方向,test.pdf,10,嵌入式软硬件方向,False,15_char_less,5_char_less,5_char_less,position_1


In [44]:
gf.add_pos()

In [45]:
gf.data_df

,text,resume_id,row,new_text,contain_date,text_length,text_length_previous,text_length_next,row_position,pos,token_words
0,李永伟,test.pdf,1,李永伟,False,5_char_less,,15_char_less,position_1,"[(李永伟, noun:personal name)]",[李永伟]
1,男（1987）,test.pdf,2,男,True,15_char_less,5_char_less,5_char_less,position_1,"[(男, distinguishing word)]",[男]
2,广东省,test.pdf,3,广东省,False,5_char_less,15_char_less,5_char_less,position_1,"[(广东省, noun:toponym)]",[广东省]
3,深圳市,test.pdf,4,深圳市,False,5_char_less,5_char_less,5_char_less,position_1,"[(深圳市, noun:toponym)]",[深圳市]
4,南山区,test.pdf,5,南山区,False,5_char_less,5_char_less,5_char_less,position_1,"[(南山区, noun:toponym)]",[南山区]
5,+86,test.pdf,6,,False,5_char_less,5_char_less,15_char_less,position_1,"[( , None)]",[ ]
6,139-2466-9071,test.pdf,7,,False,15_char_less,5_char_less,15_char_more,position_1,"[( , None)]",[ ]
7,13924669071@139.com,test.pdf,8,com,False,15_char_more,15_char_less,5_char_less,position_1,"[(com, noun)]",[com]
8,求职意向,test.pdf,9,求职意向,False,5_char_less,15_char_more,15_char_less,position_1,"[(求职, verb:noun-verb), (意向, noun)]","[求职, 意向]"
9,嵌入式软硬件方向,test.pdf,10,嵌入式软硬件方向,False,15_char_less,5_char_less,5_char_less,position_1,"[(嵌入, verb), (式, suffix), (软硬件, noun), (方向, no...","[嵌入, 式, 软硬件, 方向]"


In [31]:
content_list

,text,resume_id,row,new_text,contain_date,text_length,text_length_previous,text_length_next,row_position,pos,token_words,crf_token_word,crf_feature,section_label
0,李永伟,test.pdf,1,李永伟,False,5_char_less,,15_char_less,position_1,"[(李永伟, noun:personal name)]",[李永伟],[李永伟 noun:personal name],"[李永伟 noun:personal name, current: 5_char_less,...",I-0
1,男（1987）,test.pdf,2,男,True,15_char_less,5_char_less,5_char_less,position_1,"[(男, distinguishing word)]",[男],[男 distinguishing word],"[男 distinguishing word, current: 15_char_less,...",I-0
2,广东省,test.pdf,3,广东省,False,5_char_less,15_char_less,5_char_less,position_1,"[(广东省, noun:toponym)]",[广东省],[广东省 noun:toponym],"[广东省 noun:toponym, current: 5_char_less, previ...",I-0
3,深圳市,test.pdf,4,深圳市,False,5_char_less,5_char_less,5_char_less,position_1,"[(深圳市, noun:toponym)]",[深圳市],[深圳市 noun:toponym],"[深圳市 noun:toponym, current: 5_char_less, previ...",I-0
4,南山区,test.pdf,5,南山区,False,5_char_less,5_char_less,5_char_less,position_1,"[(南山区, noun:toponym)]",[南山区],[南山区 noun:toponym],"[南山区 noun:toponym, current: 5_char_less, previ...",I-0
5,+86,test.pdf,6,,False,5_char_less,5_char_less,15_char_less,position_1,"[( , None)]",[ ],[ None],"[ None, current: 5_char_less, previous: 5_cha...",I-0
6,139-2466-9071,test.pdf,7,,False,15_char_less,5_char_less,15_char_more,position_1,"[( , None)]",[ ],[ None],"[ None, current: 15_char_less, previous: 5_ch...",I-0
7,13924669071@139.com,test.pdf,8,com,False,15_char_more,15_char_less,5_char_less,position_1,"[(com, noun)]",[com],[com noun],"[com noun, current: 15_char_more, previous: 15...",I-0
8,求职意向,test.pdf,9,求职意向,False,5_char_less,15_char_more,15_char_less,position_1,"[(求职, verb:noun-verb), (意向, noun)]","[求职, 意向]","[求职 verb:noun-verb, 意向 noun]","[求职 verb:noun-verb, 意向 noun, current: 5_char_l...",I-0
9,嵌入式软硬件方向,test.pdf,10,嵌入式软硬件方向,False,15_char_less,5_char_less,5_char_less,position_1,"[(嵌入, verb), (式, suffix), (软硬件, noun), (方向, no...","[嵌入, 式, 软硬件, 方向]","[嵌入 verb, 式 suffix, 软硬件 noun, 方向 noun]","[嵌入 verb, 式 suffix, 软硬件 noun, 方向 noun, current...",I-0


In [6]:
gf.add_crf_feature()

In [7]:
test1 = gf.data_df

In [8]:
test1

,text,resume_id,row,new_text,contain_date,text_length,text_length_previous,text_length_next,row_position,pos,token_words,crf_token_word,crf_feature
0,李永伟,test.pdf,1,李永伟,False,5_char_less,,15_char_less,position_1,"[(李永伟, noun:personal name)]",[李永伟],[李永伟 noun:personal name],"[李永伟 noun:personal name, current: 5_char_less,..."
1,男（1987）,test.pdf,2,男,True,15_char_less,5_char_less,5_char_less,position_1,"[(男, distinguishing word)]",[男],[男 distinguishing word],"[男 distinguishing word, current: 15_char_less,..."
2,广东省,test.pdf,3,广东省,False,5_char_less,15_char_less,5_char_less,position_1,"[(广东省, noun:toponym)]",[广东省],[广东省 noun:toponym],"[广东省 noun:toponym, current: 5_char_less, previ..."
3,深圳市,test.pdf,4,深圳市,False,5_char_less,5_char_less,5_char_less,position_1,"[(深圳市, noun:toponym)]",[深圳市],[深圳市 noun:toponym],"[深圳市 noun:toponym, current: 5_char_less, previ..."
4,南山区,test.pdf,5,南山区,False,5_char_less,5_char_less,5_char_less,position_1,"[(南山区, noun:toponym)]",[南山区],[南山区 noun:toponym],"[南山区 noun:toponym, current: 5_char_less, previ..."
5,+86,test.pdf,6,,False,5_char_less,5_char_less,15_char_less,position_1,"[( , None)]",[ ],[ None],"[ None, current: 5_char_less, previous: 5_cha..."
6,139-2466-9071,test.pdf,7,,False,15_char_less,5_char_less,15_char_more,position_1,"[( , None)]",[ ],[ None],"[ None, current: 15_char_less, previous: 5_ch..."
7,13924669071@139.com,test.pdf,8,com,False,15_char_more,15_char_less,5_char_less,position_1,"[(com, noun)]",[com],[com noun],"[com noun, current: 15_char_more, previous: 15..."
8,求职意向,test.pdf,9,求职意向,False,5_char_less,15_char_more,15_char_less,position_1,"[(求职, verb:noun-verb), (意向, noun)]","[求职, 意向]","[求职 verb:noun-verb, 意向 noun]","[求职 verb:noun-verb, 意向 noun, current: 5_char_l..."
9,嵌入式软硬件方向,test.pdf,10,嵌入式软硬件方向,False,15_char_less,5_char_less,5_char_less,position_1,"[(嵌入, verb), (式, suffix), (软硬件, noun), (方向, no...","[嵌入, 式, 软硬件, 方向]","[嵌入 verb, 式 suffix, 软硬件 noun, 方向 noun]","[嵌入 verb, 式 suffix, 软硬件 noun, 方向 noun, current..."


In [9]:
import segmentation
sg = segmentation.segmentation("crf")
test = sg.predict(gf.data_df)

AttributeError: 'label_crf_classifier' object has no attribute 'tagger'

In [15]:
test

,text,resume_id,row,new_text,contain_date,text_length,text_length_previous,text_length_next,row_position,pos,token_words,crf_token_word,crf_feature,section_label
0,李永伟,test.pdf,1,李永伟,False,5_char_less,,15_char_less,position_1,"[(李永伟, noun:personal name)]",[李永伟],[李永伟 noun:personal name],"[李永伟 noun:personal name, current: 5_char_less,...",I-0
1,男（1987）,test.pdf,2,男,True,15_char_less,5_char_less,5_char_less,position_1,"[(男, distinguishing word)]",[男],[男 distinguishing word],"[男 distinguishing word, current: 15_char_less,...",I-0
2,广东省,test.pdf,3,广东省,False,5_char_less,15_char_less,5_char_less,position_1,"[(广东省, noun:toponym)]",[广东省],[广东省 noun:toponym],"[广东省 noun:toponym, current: 5_char_less, previ...",I-0
3,深圳市,test.pdf,4,深圳市,False,5_char_less,5_char_less,5_char_less,position_1,"[(深圳市, noun:toponym)]",[深圳市],[深圳市 noun:toponym],"[深圳市 noun:toponym, current: 5_char_less, previ...",I-0
4,南山区,test.pdf,5,南山区,False,5_char_less,5_char_less,5_char_less,position_1,"[(南山区, noun:toponym)]",[南山区],[南山区 noun:toponym],"[南山区 noun:toponym, current: 5_char_less, previ...",I-0
5,+86,test.pdf,6,,False,5_char_less,5_char_less,15_char_less,position_1,"[( , None)]",[ ],[ None],"[ None, current: 5_char_less, previous: 5_cha...",I-0
6,139-2466-9071,test.pdf,7,,False,15_char_less,5_char_less,15_char_more,position_1,"[( , None)]",[ ],[ None],"[ None, current: 15_char_less, previous: 5_ch...",I-0
7,13924669071@139.com,test.pdf,8,com,False,15_char_more,15_char_less,5_char_less,position_1,"[(com, noun)]",[com],[com noun],"[com noun, current: 15_char_more, previous: 15...",I-0
8,求职意向,test.pdf,9,求职意向,False,5_char_less,15_char_more,15_char_less,position_1,"[(求职, verb:noun-verb), (意向, noun)]","[求职, 意向]","[求职 verb:noun-verb, 意向 noun]","[求职 verb:noun-verb, 意向 noun, current: 5_char_l...",I-0
9,嵌入式软硬件方向,test.pdf,10,嵌入式软硬件方向,False,15_char_less,5_char_less,5_char_less,position_1,"[(嵌入, verb), (式, suffix), (软硬件, noun), (方向, no...","[嵌入, 式, 软硬件, 方向]","[嵌入 verb, 式 suffix, 软硬件 noun, 方向 noun]","[嵌入 verb, 式 suffix, 软硬件 noun, 方向 noun, current...",I-0


In [56]:
# %load classifier/label_crf_classifier.py
import os
import re
import sys
import nltk
import pickle
import numpy as np
import jieba
from random import shuffle
from sklearn.metrics import accuracy_score, confusion_matrix
from pycrfsuite import ItemSequence, Trainer, Tagger
import pandas as pd
#import generate_feature as gf

class label_crf_classifier(object):
    """Conditional Random Field model"""
    def __init__(self, stopword_path = "data/stop_words.txt"):
        self.stoplist = set(open(stopword_path).read().decode("utf8","ignore").split("\n"))

    def preprocess(self, training_data):
        self.training_df = training_data[training_data["section_label"] != ""]
        self.training_df['crf_feature'] = self.training_df['crf_feature'].map(lambda x: [y for y in x if y not in self.stoplist])
        self.x = self.training_df.groupby("resume_id")["crf_feature"].apply(list)
        self.y = self.training_df.groupby("resume_id")["section_label"].apply(list)

    def train(self, training_data, classifier_path = "classifier/cache/label_crf_classifier", c1 = 0, c2 = 10, period = 300, minfreq = 5):
        self.preprocess(training_data)
        train = Trainer()
        for i1, i in enumerate(self.x):
            train.append(ItemSequence(i), self.y[i1])
        params = {
            "c1": c1,
            "c2": c2,
            "period": period,
            "feature.minfreq": minfreq,
            "max_iterations": 1000
            # "calibration.eta": 0.05,
            # "calibration_samples": 400,
        }
        # train.select(algorithm = "l2sgd")
        train.set_params(params)
        train.train(classifier_path)
        self.tagger = Tagger()
        self.tagger.open(classifier_path)

    def save_classifier(self, classifier_path = "classifier/cache/label_crf_classifier"):
        pass

    def load_classifier(self, classifier_path = "classifier/cache/label_crf_classifier"):
        self.tagger = Tagger()
        self.tagger.open(classifier_path)

    def predict(self, test_data):
        """Input: x should be a list of strings"""
        result = self.tagger.tag(ItemSequence(test_data["crf_feature"]))
        test_data["section_label"] = result
        return test_data

    # def score(self, training_data, classifier_path="classifier/cache/label_crf_classifier", portion=0.8, c1=0, c2=10, period=300, minfreq=10):
    #     # split resume_id
    #     resume_ids = np.unique([resume['resume_id'] for resume in training_data])
    #     length = len(resume_ids)
    #     shuffle(resume_ids)
    #     train_ids = resume_ids[:int(length*portion)]
    #     test_ids = resume_ids[int(length*portion):]

    #     train_df = [resume for resume in training_data if resume['resume_id'] in train_ids]
    #     test_df = [resume for resume in training_data if resume['resume_id'] in test_ids]

    #     # train model on train_ids
    #     self.train(train_df, classifier_path=classifier_path, c1=c1, c2=c2, period=period, minfreq=minfreq)
    #     test_pred = self.predict_all(test_df)
    #     train_pred = self.predict_all(train_df)

    #     # print out result
    #     return train_pred, test_pred

# if __name__ == "__main__":
#     data = MongoRetriveData()
#     resumes = data.get_data_mongo()
#     # pickle.dump(resumes, open('./resume_data.pkl', 'wb'))
#     # resumes = pickle.load(open('./resume_data.pkl', 'rb'))
#     stopword_path = './stopword.txt'
#     model_path = './model.txt'
#     resume_data = resumes
#     clf = Crf(stopword_path, model_path, resume_data)
#     clf.CleanData()
#     clf.Fit()
#     clf.Score()
#     # result = clf.Predict(clf.data)   
#     # print result

In [57]:
crf = label_crf_classifier()

In [ ]:
crf.predict(gf.data_df)